In [8]:
import json
import re
import pickle

In [9]:
from transformers import AutoTokenizer, LlamaConfig, LlamaTokenizer,LlamaForCausalLM, LlamaTokenizer,TextDataset,DataCollatorForLanguageModeling
from peft import get_peft_model, PeftModel, PeftConfig ,LoraConfig,TaskType
from transformers import TrainingArguments, Trainer,GenerationConfig,LineByLineTextDataset
from datasets import Dataset , load_dataset
import torch
import accelerate
from transformers.generation.utils import GreedySearchDecoderOnlyOutput
import textwrap
#import bitsandbytes
from torch.nn import CrossEntropyLoss
import numpy as np
import csv

In [4]:
path="/home/vicuna-weights-7B"
tokenizer = LlamaTokenizer.from_pretrained(path)

In [1]:
new_path="/home/medical-llama/wikimed-data/wikimed-sub.txt"

with open(new_path, "r", encoding="utf-8") as file:
    medical_text = file.read()

In [6]:
import json

json_file = '/home/medical-llama/wikimed-data/wikimed.json'

data = []
with open(json_file, 'r') as f:
    for line in f:
        json_object = json.loads(line)
        data.append(json_object)

In [5]:
def clean_text(text):
    # Split text into lines
    lines = text.split('\n')

    # Remove first line (heading) and empty first line if it exists
    lines = lines[1:] if lines[0] == '' else lines[2:]

    # Remove lines containing bullets
    lines = [line for line in lines if not line.startswith("BULLET::::-")]

    # Join lines back into text
    text = '\n'.join(lines)

    text = re.sub(r'\n{2,}', '\n', text)

    return text

In [8]:
file_path_train = "/home/medical-llama/finetune-data/medical-train.txt"
with open(file_path_train, "w") as file:
    for i in range(500,25500):
        processed_text=clean_text(data[i]['text'])
        file.write(str(processed_text) + "\n")

print("List elements written to text file successfully.")

List elements written to text file successfully.


In [11]:
file_path_val = "/home/medical-llama/finetune-data/medical-val.txt"
with open(file_path_val, "w") as file:
    for i in range(0,200):
        processed_text=clean_text(data[i]['text'])
        file.write(str(processed_text) + "\n")

print("List elements written to text file successfully.")

List elements written to text file successfully.


In [12]:
with open(file_path_val, "r", encoding="utf-8") as file:
    medical_text = file.read()

tokenized_text = tokenizer.encode(medical_text)
len(tokenized_text)

346258

In [87]:
av=[]
for i in range(len(data)):
    av.append(len(data[i]['text'].split(" ")))

print(sum(av)/len(av))

592.7705923001489


In [64]:
print(clean_text(data[1]['text']))

Etorphine (M99) is a semi-synthetic opioid possessing an analgesic potency approximately 1,000–3,000 times that of morphine. It was first prepared in 1960 from oripavine, which does not generally occur in opium poppy extract but rather the related plants "Papaver orientale" and "Papaver bracteatum". It was later reproduced in 1963 by a research group at MacFarlan Smith in Gorgie, Edinburgh, led by Kenneth Bentley. It can also be produced from thebaine.
Etorphine is available legally only for veterinary use and is strictly governed by law. It is often used to immobilize elephants and other large mammals. Diprenorphine (Revivon) is an opioid receptor antagonist that can be administered in proportion to the amount of etorphine used (1.3 times) to reverse its effects. Veterinary-strength etorphine is fatal to humans. For this reason the package as supplied to vets always includes the human antidote as well as etorphine.
The human antidote is generally naloxone, not diprenorphine, and is al

In [40]:
title_list = []
for i in range(len(data)):
    title_list.append(data[i]['title'])

In [41]:
len(title_list)

393618

In [18]:
clean_text(a)

'\nPolypterus is a genus of freshwater fish in the bichir family (Polypteridae) of order Polypteriformes. The type species is the Nile bichir ("P. bichir"). Fishes in this genus live in various areas in Africa. "Polypterus" is the only known vertebrate to have lungs, but no trachea.\n\nThe etymology of the genus name derives from a combination of the Greek prefix πολυ-, "poly-" (many) and the root word πτερον, "pteron" (wing or fin) – "many fins".\n\nIn shallow water, "Polypterus" inhales primarily through its spiracle (blowhole). Exhalation is powered by muscles in the torso. During exhalation, the bony scales in the upper chest become indented. When the muscles are relaxed, the bony scales spring back into position, generating negative pressure within the torso, resulting in a rapid intake of air through the spiracle. The air is nearly sufficient to fill the lungs. This is followed by one cycle of buccal (mouth) pumping, which "tops off" the lungs, with the surplus air from the bucca

In [42]:
title_list

['Polypterus',
 'Etorphine',
 'Leeza Gibbons',
 'USS Niagara (1813)',
 'Meramec State Park',
 'Berosus',
 'Nutritional genomics',
 'Alluaudia',
 'Goblet cell',
 'Lobos Island',
 'Interstate 140 (North Carolina)',
 'Ulvaceae',
 'Carya ovata',
 'Interstate 540 and North Carolina Highway 540',
 'Lesser bandicoot rat',
 'Carole Meredith',
 'Fumagillin',
 'Tubifex tubifex',
 'Víctor Trujillo',
 'Sanguinaria',
 'Breaking Open the Head',
 'Environmentalism',
 'Common mudpuppy',
 'The Terminal Experiment',
 'Logan Mankins',
 'Oriental cockroach',
 'Arizona Stadium',
 'Mosh (wrestler)',
 'German cockroach',
 'James Collip',
 'Thalamotomy',
 'Self-control',
 'Ceratonova shasta',
 'Anasa tristis',
 'Marine iguana',
 'Dracaena trifasciata',
 'Sirene',
 'Window film',
 'Murder of Marcia Trimble',
 'Thừa Thiên-Huế Province',
 'Aphelandra',
 'Acalypha',
 'Apostatic selection',
 'Tabernaemontana',
 'Curare',
 'Stereopsis',
 'Pogonomyrmex',
 'Frederick Ludlow',
 'List of birds of Missouri',
 'Gordon Ba

In [29]:
# Open the JSON file
json_file="/home/medical-llama/Pubmedqa/pubmedqa/data/test_set.json"


with open(json_file, 'r',encoding='utf-8') as f:
    json_dict=json.load(f)


In [42]:
with open("/home/medical-llama/Pubmedqa/pubmedqa/data/pubmedqa_test.csv", "w", newline="") as csv_file:
    writer = csv.writer(csv_file)

    # Write the CSV header
    writer.writerow(["ID", "Question", "Context", "Label"])
    for i in json_dict.keys():
        writer.writerow([i, json_dict[i]['QUESTION'], " ".join(json_dict[i]['CONTEXTS']), json_dict[i]['final_decision']])
    
    


In [30]:
json_dict.keys()

dict_keys(['12377809', '26163474', '19100463', '18537964', '12913878', '12765819', '25475395', '19130332', '9427037', '24481006', '8165771', '22680064', '22540518', '20629769', '21726930', '21481154', '22902073', '26370095', '18041059', '15041506', '11146778', '27281318', '21645374', '9465206', '25887165', '15995461', '21850494', '19106867', '21342862', '24352924', '16147837', '26879871', '15918864', '22075911', '11035130', '21228436', '11833948', '17682349', '17355582', '15597845', '10158597', '27549226', '26348845', '25588461', '23359100', '26548832', '25756710', '20297950', '24622801', '9722752', '20577124', '19027440', '18239988', '27858166', '27050489', '16266387', '27287237', '11079675', '10331115', '18594195', '22497340', '16769333', '20571467', '12094116', '17276182', '26419377', '23810330', '15151701', '23736032', '28143468', '23495128', '12121321', '18570208', '28707539', '22117569', '18783922', '15528969', '19482903', '11977907', '17306983', '24318956', '22266735', '22453060

In [41]:
for i in json_dict.keys():
    writer.writerow([i, json_dict[i]['QUESTION'], " ".join(json_dict[i]['CONTEXTS']), json_dict[i]['final_decision']])
    
    

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 for i in json_dict.keys():                                                                   │
│ ❱ 2 │   writer.writerow([i, json_dict[i]['QUESTION'], " ".join(json_dict[i]['CONTEXTS']), js     │
│   3                                                                                              │
│   4                                                                                              │
│   5                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: I/O operation on closed file.

In [43]:
import pandas as pd

# Open the CSV file
df = pd.read_csv("/home/medical-llama/Pubmedqa/pubmedqa/data/pubmedqa_test.csv")

In [44]:
df.head()

,ID,Question,Context,Label
0,12377809,Is anorectal endosonography valuable in dysche...,Dyschesia can be provoked by inappropriate def...,yes
1,26163474,Is there a connection between sublingual varic...,Sublingual varices have earlier been related t...,yes
2,19100463,Is the affinity column-mediated immunoassay me...,Tacrolimus is a potent immunosuppressive drug ...,yes
3,18537964,Does a physician's specialty influence the rec...,To determine the impact of a physician's speci...,yes
4,12913878,Locoregional opening of the rodent blood-brain...,Nd:YAG laser-induced thermo therapy (LITT) of ...,yes


In [45]:
df['Label'].value_counts()

Label
yes      276
no       169
maybe     55
Name: count, dtype: int64

In [46]:
df['Question'][2]

'Is the affinity column-mediated immunoassay method suitable as an alternative to the microparticle enzyme immunoassay method as a blood tacrolimus assay?'